# Training Arithmetic

**References**
- *Transformer Math 101: [Blog](https://blog.eleuther.ai/transformer-math/)*
- *Transformer Inference Arithmetic: [Blog](https://kipp.ly/transformer-inference-arithmetic/)*

## Training Arithmetic

#### Compute Requirements

Cost to train a transformer model:
$$C \approx \tau T = 6 PD$$

where:
- $C$ is the compute required to train the transformer model, in total floating point operations
- $C = C_{forward} + C_{backward}$
- $C_{forward} \approx 2PD$
- $C_{backward} \approx 4PD$
- $\tau$ is the aggregate throughput of your hardware setup $\big( \tau = \text{(No. GPUs)} \times \text{(Actual FLOPs/GPU)} \big)$, in FLOPs
- $T$ is the time spent training the model, in seconds
- $P$ is the number of parameters in the transformer model
- $D$ is the dataset size in tokens

*These equations are proposed and experimentally validated in OpenAI’s scaling laws paper and DeepMind’s scaling laws paper.*

$C$ is a measure of total compute and can be measured by many units such as:
- FLOP-seconds, which is in units of $\big[ \frac{\text{Floating Point Operations}}{\text{Second}}\big] \times [\text{Seconds}]$
- GPU-hours, which is in units of $[\text{No. GPUs}] \times [\text{Hours}]$
- Scaling laws papers tend to report values in PetaFLOP-days, or $10^{15} \times 24 \times 3600$ total floating point operations.


#### Parameter vs Dataset Tradeoffs

According to “Chinchilla scaling laws”, compute optimal language model has a **number of parameters** and a **dataset size** that satisfies the approximation $D = 20P$. This is optimal in one very specific sense: in a resource regime where using 1,000 GPUs for 1 hour and 1 GPU for 1,000 hours cost you the same amount, if your goal is to maximize performance while minimizing the cost in GPU-hours to train a model you should use the above equation.

#### Engineering Takeaways for Compute Costs

Computing costs for transformers are typically listed in GPU-hours or FLOP-seconds.
- GPT-NeoX achieves 150 TFLOP/s/A100 with normal attention and 180 TFLOP/s/A100 with Flash Attention. This is in line with other highly optimized libraries at scale, for example Megatron-DS reports between 137 and 163 TFLOP/s/A100.
- As a general rule of thumb, you should always be able to achieve approximately 120 TFLOP/s/A100. If you are seeing below 115 TFLOP/s/A100 there is probably something wrong with your model or hardware configuration.
- With high-quality interconnect such as InfiniBand, you can achieve linear or sublinear scaling across the data parallel dimension (i.e. increasing the data parallel degree should increase the overall throughput nearly linearly).

#### Model Weights

Most transformers are trained in mixed precision, either fp16 + fp32 or bf16 + fp32.
![mixed precision](./images/dl-precisions.png)

These numbers refer to the size in bits a single parameter requires. Since there are 8 bits in a Byte, we divide this number by 8 to see how many Bytes each parameter requires:
- In int8, $\text{memory}_{model} = (1 \text{byte} / \text{param}) \cdot (\text{No. params})$
- In fp16 and bf16, $\text{memory}_{model} = (2 \text{bytes} / \text{param}) \cdot (\text{No. params})$
- In fp32, $\text{memory}_{model} = (4 \text{bytes} / \text{param}) \cdot (\text{No. params})$


#### Total Inference Memory

In addition to the memory needed to store the model weights, there is also a small amount of additional overhead during the actual forward pass which is typically irrelevant to determining the largest model that will fit on your GPU.

In total, a good heuristic answer for “will this model fit for inference” is:
$$\text{Total Memory}_{Inference} \approx (1.2) \times \text{Model Memory}$$


### Training

#### Model Parameters

Models can be trained in pure fp32 or fp16:
- Pure fp32, $\text{memory}_{model} = (4 \text{bytes} / \text{param}) \cdot (\text{No. params})$
- Pure fp16, $\text{memory}_{model} = (2 \text{bytes} / \text{param}) \cdot (\text{No. params})$

In addition to the common model weight datatypes discussed in Inference, training introduces mixed-precision training such as Automatic Mixed Precision (AMP). This technique seeks to maximize the throughput of GPU tensor cores while maintaining convergence. The modern DL training landscape frequently uses mixed-precision training because: 
1. fp32 training is stable, but has a high memory overhead and doesn’t exploit NVIDIA GPU tensor cores, and
2. fp16 training is stable but difficult to converge.

Note that mixed-precision requires an fp16/bf16 and fp32 version of the model to be stored in memory, requiring:
- Mixed-precision (fp16/bf16 and fp32), $\text{memory}_{model} = (2 \text{bytes} / \text{param}) \cdot (\text{No. params})$

plus an additional size $(4 \text{bytes} / \text{param}) \cdot (\text{No. params})$ copy of the model that we count within our optimizer states.


#### Optimizer States

Adam is magic, but it’s highly memory inefficient. In addition to requiring you to have a copy of the model parameters and the gradient parameters, you also need to keep an additional three copies of the gradient parameters:
- For vanilla AdamW, $\text{memory}_{optimizer} = (12 \text{bytes} / \text{param}) \cdot (\text{No. params})$
    - fp32 copy of parameters: 4 bytes/param
    - Momentum: 4 bytes/param
    - Variance: 4 bytes/param
- For 8-bit optimizers like [bitsandbytes](https://github.com/TimDettmers/bitsandbytes), $\text{memory}_{optimizer} = (6 \text{bytes} / \text{param}) \cdot (\text{No. params})$
    - fp32 copy of parameters: 4 bytes/param
    - Momentum: 1 byte/param
    - Variance: 1 byte/param
- For SGD-like optimizers with momentum, $\text{memory}_{optimizer} = (8 \text{bytes} / \text{param}) \cdot (\text{No. params})$
    - fp32 copy of parameters: 4 bytes/param
    - Momentum: 4 bytes/param

#### Gradients

Gradients can be stored in fp32 or fp16 (Note that the gradient datatype often matches the model datatype. We see that it therefore is stored in fp16 for fp16 mixed-precision training), so their contribution to memory overhead is given by:
- In fp32, $\text{memory}_{gradients} = (4 \text{bytes} / \text{param}) \cdot (\text{No. params})$
- In fp16, $\text{memory}_{gradients} = (2 \text{bytes} / \text{param}) \cdot (\text{No. params})$


#### Activations and Batch Size

Modern GPUs are typically bottlenecked by memory, not FLOPs, for LLM training. Therefore activation recomputation/checkpointing is an extremely popular method of trading reduced memory costs for extra compute costs. Activation recomputation/checkpointing works by recomputing activations of certain layers instead of storing them in GPU memory. The reduction in memory depends on how selective we are when deciding which layers to clear, but Megatron’s selective recomputation scheme is depicted in the figure below:

![Megatrons selective recomputation scheme](./images/Megatrons-selective-recomputation-scheme.png)

Where the dashed red line indicates the memory capacity of an A100-80GB GPU, and “present work” indicates the memory requirements after applying selective activation recomputation.

The basic equation from [Reducing Activation Recomputation in Large Transformer Models](https://arxiv.org/abs/2205.05198) gives the memory required to store activations for a transformer model:
$$\text{memory}_{activations}^{No Recomputation} = sbhL(10 + \frac{24}{t} + 5 \frac{a \cdot s}{h \cdot t}) \text{bytes}$$
$$\text{memory}_{activations}^{Selective Recomputation} = sbhL(10 + \frac{24}{t}) \text{bytes}$$
$$\text{memory}_{activations}^{Full Recomputation} = 2 \cdot sbhL \ \text{bytes}$$

where:
- $s$ is the sequence length, in tokens
- $b$ is the batch size per GPU
- $h$ is the dimension of the hidden size within each transformer layer
- $L$ is the number of layers in the transformer model
- $a$ is the number of attention heads in the transformer model
- $t$ is the degree of tensor parallelism being used (1 if not)
- We assume no sequence parallelism is being used
- We assume that activations are stored in fp16

The additional recomputation necessary also depends on the selectivity of the method, but it’s bounded above by a full additional forward pass. Hence the updated cost of the forward pass is given by:

$$2PD \leq C_{forward} \leq 4PD$$


#### Total Training Memory

Therefore, a good heuristic answer for “will this model fit for training” is:

$$\text{Total Memory}_{Training} = \text{Model Memory} + \text{Optimizer Memory} + \text{Activation Memory} + \text{Gradient Memory}$$


### Distributed Training

#### Sharded Optimizers

The massive memory overheads for optimizers is the primary motivation for sharded optimizers such as [ZeRO](https://arxiv.org/abs/1910.02054) and [FSDP](https://engineering.fb.com/2021/07/15/open-source/fsdp/). Such sharding strategies reduce the optimizer overhead by a factor of **No. GPUs**, which is why a given model configuration may fit at large scale but OOM at small scales. If you’re looking to calculate the memory overhead required by training using a sharded optimizer, you will need to include the equations from the figure below. For some sample calculations of sharded optimization, see the following figure from the ZeRO paper (Note that $P_{os} P_{os + g}$ and $P_{os + g + p}$ are commonly denoted as ZeRO-1, ZeRO-2, ZeRO-3, respectively. ZeRO-0 commonly means  "Zero disabled")

![Sharded Optimizers](./images/zero-fig-Sharded-Optimizers.png)

In the language of this blog post (assuming mixed-precision and the Adam optimizer):
- For ZeRO-1
$$\text{Total Memory}_{Training} \approx \text{Model Memory} + \frac{\text{Optimizer Memory}}{\text{(No. GPUs)}} + \text{Activation Memory} + \text{Gradient Memory}$$
- For ZeRO-2
$$\text{Total Memory}_{Training} \approx \text{Model Memory} + \text{Activation Memory} + \frac{\text{Optimizer Memory} + \text{Gradient Memory}}{\text{(No. GPUs)}}$$
- For ZeRO-3
$$\text{Total Memory}_{Training} \approx \text{Activation Memory} + \frac{\text{Model Memory} + \text{Optimizer Memory} + \text{Gradient Memory}}{\text{(No. GPUs)}}$$

Where (DP Degree) is just (No. GPUs) unless pipeline and/or tensor parallelism are applied.

Note that ZeRO-3 introduces a set of live parameters. This is because ZeRO-3 introduces a set of config options (**stage3_max_live_parameters, stage3_max_reuse_distance, stage3_prefetch_bucket_size, stage3_param_persistence_threshold**) that control how many parameters are within GPU memory at a time (larger values take more memory but require less communication). Such parameters can have a significant effect on total GPU memory.

Note that ZeRO can also partition activations over data parallel ranks via **ZeRO-R**. This would also bring the $\text{memory}_{activations}$ above the tensor parallelism degree $t$. For more details, read the associated [ZeRO paper](https://arxiv.org/abs/1910.02054) and [config options](https://www.deepspeed.ai/docs/config-json/#activation-checkpointing) (note in GPT-NeoX, this is the `partition_activations` flag). If you are training a huge model, you would like to trade some memory overhead for additional communication cost, and activations become a bottleneck. As an example of using ZeRO-R along with ZeRO-1:

$$\text{Total Memory}_{Training} \approx \text{Model Memory} + \frac{\text{Optimizer Memory}}{\text{(No. GPUs)}} + \frac{\text{Activation Memory}}{\text{(Tensor-Parallel-Size)}} + \text{Gradient Memory}$$




#### 3D Parallelism

Parallelism for LLMs comes in 3 primary forms:
- **Data parallelism**: Split the data among (possibly model-parallel) replicas of the model
- **Pipeline or Tensor/Model parallelism**: These parallelism schemes split the parameters of the model across GPUs. Such schemes require significant communication overhead, but their memory reduction is approximately:
$$\text{memory}_{model}^{w/parallelism} \approx \frac{\text{Model Memory}}{(\text{Pipe-Parallel-Size}) \times (\text{Tensor-Parallel-Size})}$$
$$\text{memory}_{gradients}^{w/parallelism} \approx \frac{\text{Gradient Memory}}{(\text{Pipe-Parallel-Size})}$$

Note that this equation is approximate due to the facts that 
1. pipeline parallelism doesn’t reduce the memory footprint of activations,
2. pipeline parallelism requires that all GPUs store the activations for all micro-batches in-flight, which becomes significant for large models, and
3. GPUs need to temporarily store the additional communication buffers required by parallelism schemes.


#### Sharded Optimizers + 3D Parallelism

When ZeRO is combined with tensor and/or pipeline parallelism, the resulting parallelism strategy forms a mesh like the following:

![Sharded Optimizers and 3D Parallelism](./images/Sharded-Optimizers-and-3D-Parallelism.png)

As an important aside, the DP degree is vital for use in calculating the global batch size of training. The data-parallel degree depends on the number of complete model replicas:
$$\text{DP Degree} = \frac{\text{No. GPUs}}{(\text{Pipe-Parallel-Size}) \times (\text{Tensor-Parallel-Size})}$$

Pipeline parallelism and tensor parallelism are compatible with all stages of ZeRO. However, it's difficult to maintain efficiency when combining pipeline parallelism with ZeRO-2/3's gradient sharding (Because ZeRO-2 shards the gradients, but pipeline parallelism accumulates them. It's possible to carefully define a pipeline schedule and overlap communication to maintain efficiency, but it's difficult to the point that DeepSpeed currently [forbids it](https://github.com/microsoft/DeepSpeed/blob/v0.10.1/deepspeed/runtime/pipe/engine.py#L71)).

Tensor parallelism, however, is complementary to all stages of ZeRO because on each rank:
- ZeRO-3 gathers the full layer **parameters** from other ranks, processes a **full** input on the now-local full layer, then frees the memory that was allocated to hold the remote ranks' parameters.
- Tensor Parallelism gathers the remote **activations** for the local input from other ranks, processes a **partition** of the input using the local layer partition, then sends the next layer's activations to remote ranks

For the majority of Eleuther's work, we train with pipeline and tensor parallelism along with ZeRO-1. This is because we find ZeRO-3 to be too communication-heavy for our hardware at large scales, and instead use pipeline parallelism across nodes along with tensor parallelism within nodes.

Putting everything together for a typical 3D-parallel ZeRO-1 run with activation partitioning:
$$\text{Total Memory}_{Training} \approx \frac{\text{Model Memory}}{(\text{Pipe-Parallel-Size}) \times (\text{Tensor-Parallel-Size})} + \frac{\text{Optimizer Memory}}{\text{(No. GPUs)}} + \frac{\text{Activation Memory}}{\text{(Tensor-Parallel-Size)}} + \frac{\text{Gradient Memory}}{\text{(Pipe-Parallel-Size)}}$$



## Inference Arithmetic

### KV Cache

*kv cache explains the performance improvement of caching self-attention vectors as a part of inferencing, as well as the possible tradeoffs and capacity costs*

With the computed $k$, $v$ values, we can save quite a bit of computation at the cost of some storage. Per token, the number of bytes we store is
$$2 \cdot 2 \cdot n_{layers} \cdot n_{heads} \cdot d_{head}$$

The first factor of 2 is to account for the two vectors, $k$ and $v$. We store that per each layer, and each of those values is a $n_{heads} \times d_{head}$ matrix. Then multiply by 2 again for the number of bytes (assuming 16-bit formats throughout the inference arithmetic).

The weights that we multiply by the token embeddings are $\mathit{W}_k, \mathit{W}_v \in \mathbb{R}^{d_{model} \times d_{model}}$ and then each token embedding is $t_e \in \mathbb{R}^{1 \times d_{model}}$. So then the flops to compute $k$ and $v$ for all our layers is
$$2 \cdot 2 \cdot n_{layers} \cdot d_{model}^2$$

We multiply $t_e$ by $\mathit{W}_k$, which takes $$2 \cdot d_{model}^2$$ flops. We have another factor of 2 as we do that twice, once each for $k$ and $v$ and then repeat for $n_{layers}$.

For a 52B parameter model (taking Anthropic's, where $d_{model} = 8192$ and $n_{layers} = 64$). The flops are
$$2 \cdot 2 \cdot 64 \cdot 8192^2 = 17,179,869,184$$

Say we have an A100 GPU, which does $312e12$ flops per second and $1.5e12$ bytes per second of memory bandwidth. The following are numbers for just the kv weights and computations.
$$\text{memory} = 2 \cdot 2 \cdot n_{layers} \cdot d_{model}^2 \div 1.5e12$$
$$\text{compute} = 2 \cdot 2 \cdot n_{layers} \cdot d_{model}^2 \div 312e12$$

None of the model architecture matters anymore — we get a distinct ratio here of 208 given this hardware specification. This means that if we're going to compute kv for one token, it'll take the same amount of time to compute for up to 208 tokens! Anything below, we're memory bandwidth bound. Above, flops bound. 

If we used the rest of our weights to do a full forwards pass on our context, it's also 208 (both the numerator and denominator get a factor of 6 added). The intersection of the below diagram is at 208, though in reality the memory line does have a slight slope due to memory cost of intermediate calculations.

![Memory Compute Intersection](./images/Memory-Compute-Intersection.png)

For a 52B model full forwards pass, that's $12 \cdot 2 \cdot n_{layers} \cdot d_{model}^2 / 1.5e12 \approx 69 \text{milliseconds}$ for upto 208 tokens (in practice, we'd use four GPUs in parallel so it would actually be ~17 milliseconds). If we had 416 (double) tokens in the context, then it would take twice as long, and 312 tokens would take 1.5 times as long.

Calculating for a kv cache token is exactly 1/6th of the compute of passing the token through the model. In general, these forwards passes (what we experience in getting logits, embeddings and training) are very cheap because of the parallelism that is possible as opposed to sampling where we're forced to read through all the weights for each token and do the autoregression.

This doesn't mean that 1/6th of the time is saved! Let's assume we are flops bound. Then at each sample step, we save $2 \cdot 2 \cdot n_{tokens} \cdot n_{layers} \cdot d_{model}^2 \div 312e12$ flops while the decoding steps costs $2 \cdot 12 \cdot n_{layers} \cdot d_{model}^2 \div 312e12$. Thus at each step we save 1/6 of the flops time multiplied by the number of tokens in our sequence - which increases as we sample tokens. It is the case that without a kv cache, sampling would be quadratic in time complexity as we increase the number of tokens.

This is not the whole story (given overheads and tradeoffs associated with storing this cache). If we're serving small batches we may be memory bandwidth bound rather than flops, in which case we won't even want to use the past cache and will instead happily do recomputations, spending the flops (we'll already be paying the memory cost to do our sampling).


**Flops vs Memory Boundedness**

In optimizing deep learning models, especially transformers, we constantly encounter a trade-off between computational power (FLOPs) and memory access speed. While performing calculations (FLOPs), the model also needs to load data (weights) from memory, which takes time. Ideally, these processes would happen simultaneously. However, limitations can arise. A FLOP-bound scenario means calculations are faster than memory access, leading to idle waiting for data. Conversely, a memory-bound scenario means data access is slower than calculations, resulting in unused processing power.

### Capacity

*Capacity takes the storage cost of kv cache and connects it to the storage cost of model weights and what capacity means for performance.*

The size of the weights for a 52B model is $52e12 \cdot 2 = 104e12 \ \text{bytes} \approx 104\text{GB}$. Considering a A100 GPU with 40GB capacity we would need 3 GPUs, which leaves us with 16GB for kv cache after loading the 104GB model:
$$2 \cdot n_{layers} \cdot n_{heads} \cdot d_{head} \cdot 2 = 4 \cdot 64 \cdot 8192 = 2,097,152 \ \text{bytes} \approx 0.002 \text{GB}$$
And then we'd do $16/0.002 \approx 8000$ tokens can fit into our kv cache with this GPU set up, or that we could do up to a batch size 4 where each request has up to 2048 tokens (and higher sizes for less tokens).

This sucks because we would like to be able to do higher batch sizes, but are capacity limited! Higher batch sizes are more efficient in terms of how much GPU time it takes to process the same request. On the other hand, at batch sizes this low we're bound to be memory bound, and should forego the kv cache and just pay the flops cost instead.

For four GPUs, we'd get $56/0.002 \approx 23000$. We definitely want to go for the four GPUs since we'll want to be able to do higher batch sizes, and it's silly to to divide powers of two over three GPUs. But it's not just batch size! If we have high volume, then we'd have multiple instances of our models. We approximately want each instance to be able to do as large as a batch size as possible, as we pay the cost of storing the weights anyway.


### Model Parallelism

*Model Parallelism builds up an understanding specifically of tensor parallelism to clearly identify the cost of communication.*

**Model Parallelism**
- Splits a large model across multiple accelerators (GPUs) to distribute the computational load.
- This allows training models that are too big to fit on a single GPU.
**Benefits**
- Divides the cost of calculations (FLOPs) and memory usage among the accelerators.

**Two common approaches to Model Parallelism**
1. Tensor (model) parallelism:
- Splits the model itself into pieces, with each accelerator working on its assigned portion.
- Requires communication between accelerators to synchronize the training process.
- The cost of communication scales with the number of accelerators (N). More accelerators mean more data to transfer between them.

1. Pipeline parallelism:
- Divides the training process into stages, with each accelerator handling a specific stage.
- More efficient in terms of communication compared to model parallelism.
- But can lead to underutilized accelerators if the workload isn't balanced perfectly.

For attention the parallelism is intuitive from the fact that we have multiple heads. We go through most of the attention layer without communication because our attention heads are concatenated to multiply by $\mathit{W}_o$. After we multiply by $v$, we multiply the result by our shard of $\mathit{W}_o$ to get a shard of $o_s \in \mathbb{R}^{d_{model} \times n_{heads}/N}$. Then each accelerator will communicate its own shard to all the others, and all the others will communicate their shards back. This is $(N-1) d_{model} / N$ of comms cost. Each accelerator will do an even share of the addition to get the output projection, then do the same communication they did last time and the individual hosts will do the concatenation (approximately instant). The kv cache is split across heads by GPU.


### Latency Calculations

*Latency Calculations pulls understanding from other concepts to create equations that serve as floorlines for inference speed.*

Our latency calculations are mostly about the flops vs memory boundedness. If we have a small number of multiplies to do per parameter, then maybe we'll be throttled by memory bandwidth. Flops are increased by both batch size and number of parameters, while memory is only increased by number of parameters.

For comms, it's not about boundedness, but rather about adding a latency term and a throughput term (the 300GB/s). Something tricky about the latency side is that it's not reported, so the best I can do is guess "approximately small", which is approximately 8 microseconds per message sent as found in this [Citadel paper](https://arxiv.org/pdf/1804.06826) but it's for V100 NVLink.

Because of the compute factors, to calculate the latency of a single token decoding step we'd have two formulas - one for memory bandwidth bound (small batch) and another for flops bound (large batch). For large batch, we'll drop the latency factor for communications.

Equations for a small batch (say 1, so we can drop the batch factor) would be; (where $N$ is the number of accelerators and $P$ is the number of parameters and $b$ is "byte" as a unit)
$$\text{compute} = \frac{2 \cdot P \cdot b}{N \cdot A_{bm} b s^{-1}}$$
$$\text{comms} = 4 \cdot n_{layers} \cdot 8 \mu s$$

There is $2 \cdot P$ because we need to pass all the parameters through the memory, and each parameter is two bytes.
$A_{bm}$ is the accelerator memory bandwidth, and this cost is split across accelerators.
For comms, we have $4 \cdot n_{layers}$ communications per layer, and the latency per each request.
Comms will usually come out to be relatively small so for the compute bound case we won't need to pay attention to it anyway. There's also a throughput cost in comms which also rounds away.

There's another sometimes-significant factor here which is the read time for the kv cache, which I'll leave out of the equation now since it depends on number of context tokens, which can even vary within a batch and total number of tokens we want to sample. This would be calculated as memory bandwidth time. Another missing memory bandwidth time is the read of the unembeddings to calculate logits at each sampling step, which is $\in \mathbb{R}^{d_{model} \times n_{vocab}}$.

As previously mentioned, the memory does not actually stay constant, rather some additional memory is used per batch for intermediate activations. The reason we don't factor this in is simply because it's hard to count as it varies a lot by the software stack, compiler optimisations, etc.

For large batches (say 512), where $B$ is the batch size:
$$\text{compute} = B \frac{2 \cdot P \cdot \text{FLOP}}{N \cdot A_{f} \text{FLOP} s^{-1}}$$
$$\text{comms} = B \cdot \frac{2 \cdot n_{layers} \cdot 4 \cdot d_{model} \cdot b}{A_c b s^{-1}}$$

Where $A_f$ is the flops of the accelerator and $A_c$ is the comms bandwidth.
We do $2 \cdot P$ flops of operations, which can be intuited by the fact that we matmul through all the parameters.

For comms, we see the four (I'll round that $N-1$ factor to $N$) communications each of a $d_{model}$ size vector per layer as explained in the model parallelism section. We swapped out the latency calculation for a throughput one. Then it's all divided by the comms bandwidth.

Let's play with a larger model, a Gopher sized 260B model on 16 GPUs. For a small batch, it's 22 ms per token generated. The throughput cost for the comms which we can calculate with the equation for large batch is approximately 35 microseconds, assuring us that it was safe to drop.
$$\text{compute} = \frac{2 \cdot P}{N \cdot A_{bm}} = \frac{2 \cdot 260e9}{16 \cdot 1.5e12} \approx 0.0217 \approx 22ms$$
$$\text{comms} = 2 \cdot 4 \cdot n_{layers} \cdot 8 \mu s = 4 \cdot 80 \cdot 8 \mu s = 2560 \mu s \approx 3ms$$

For a large batch of 512, for a total of 53 ms per token generated (per batch, so in the 62ms 512 tokens are generated). The latency cost on comms here would've also been 3ms (latency is not multiplied by batch as the message can be prepared together) which is somewhat significant to drop but it's fine if we assuming parallel comms and compute.
$$\text{compute} = B \cdot \frac{2 \cdot P}{N \cdot A_{f}} = 512 \cdot \frac{2 \cdot 260e9}{16 \cdot 312e12} \approx 0.053 \approx 53ms$$
$$\text{comms} = B \cdot \frac{2 \cdot 4 \cdot n_{layers} \cdot d_{model}}{A_c} = 512 \cdot \frac{8 \cdot 80 \cdot 16384}{300e9} \approx 18ms$$

The higher value between the comms and compute is taken as we're assuming that it's parallel. Thus, we would want to avoid having comms being greater than compute (this is the mechanism that prevents us from approaching latency zero as we insert more chips, eventually the comms will start taking more and more time). It's not guaranteed that all systems will do this in parallel, and certainly not perfectly in parallel.

These numbers are definitely much lower than what we can get with real sand, as it assumes optimal hardware usage, doesn't factor in softmaxes, assumes zero comms latency and ignores many other smaller factors. Nonetheless, all the reasoning behind this math is useful for thinking about where to go optimise performance what deltas incoming optimisations will cause.


### Batch Sizes

*Batch Sizes discusses what impact batch size has on performance and what sizes may be optimal.*

In the previous section, we have two calculations for when something is memory bandwidth bound versus flops bound. To figure out which is at play we can compare these numbers;
$$\text{mem bandwidth time} = \frac{2 \cdot P}{N \cdot A_{bm}}$$
$$\text{flops time} = B \cdot \frac{2 \cdot P}{N \cdot A_{f}}$$

We're dealing with the same ratio we found in the kv cache section. The min batch size for memory bandwidth bound is $A_{bw}/A_f = 208$. If we have the load to do it, we prefer flops bound as it's more compute efficient. Though it's also the case that if we're flops bound, making the batch size larger doesn't mean anything is getting faster.

To calculate when the capacity goes from mostly kv cache to mostly weights is trivial, and also isn't a binary in the same way (nothing special happens when your kv cache starts taking up more memory than your weights). Nothing special really happens with comms either. At some point in increasing the batch size, the throughput starts dwarfing the latency so we dropped that factor. As observed previously, the latency becomes insignificant much later (our 512 batch on 52B communication cost was still 11% latency).

Something oversimplified about comms is that it happens at four different steps, which means we don't just want our compute time to be longer than our comms time, we want it to be the case at each step (if we can parallelise the compute and comms). For that, we have a weirder ratio: flops per byte of comms. Here's a nice chart of our computations, which will also be useful in the section below.

![flops per byte of comms](./images/flops-per-byte-of-comms.png)

$312e12 / 300e9 = 1040$, which is our flops per byte of comms for our A100s. We want the values in the last row to be larger than our hardware flops per byte so that we stay flops bound (assuming we are not memory bound here). For any model with an embedding dimension over 1024 (per chip), we're safe! For 512, it's a little awkward.

A low-load API may result in smaller batch sizes, leading to reasonable decisions like dropping the kv cache. If an API had the load for large batches it would probably want to serve the lowest batch size that gets flop bound even if there is capacity left so that it could optimise for per-request-latency.


### Flops Counting

*Flops Counting steps through the transformer blocks and identifies which operations meaningfully contribute to flops speed.*

Previously: "we do $2 \cdot P$ flops of operations, which can be intuited by the fact that we matmul through all the parameters." This is correct reasoning, but we can break it down by walking through all the transformer steps and check that we get $2P$.

The following calculations are per token, per layer. I describe $\mathit{W}_q, \mathit{W}_k, \mathit{W}_v \in \mathbb{R}^{d_{model} \times d_{model}}$ where it's more accurate to say we have $\mathit{W}_q^i, \mathit{W}_k^i, \mathit{W}_v^i \in \mathbb{R}^{d_{model} \times d_{head}}$ where $i$ goes up to $n_{heads}$. But for the sake of calculating latency, I simplify $\mathit{W}_q, \mathit{W}_k, \mathit{W}_v$ to include all the heads.

**Computing qkv**
- Multiply $t_e \in \mathbb{R}^{1 \times d_{model}}$ by $\mathit{W}_q, \mathit{W}_k, \mathit{W}_v \in \mathbb{R}^{d_{model} \times d_{model}}$
- Flop count: $2 \cdot 3 \cdot d_{model}^2$

**Calculate z**
- This is softmax $((q \cdot k) \div \sqrt{d_{head}}) \cdot v = z$
- No matrices are multiplied, the number of flops is some factor of $d_{model}$

**Multiply by the output projection matrix**
- Multiply $\mathit{W}_o \in \mathbb{R}^{d_{model} \times d_{model}}$, by $z \in \mathbb{R}^{d_{model} \times 1}$
- Flop count: $2 \cdot d_{model}^2$

**Feed-forward**
- We have our MLP weights $\mathit{W}_1 \in \mathbb{R}^{4 \times d_{model}}$, $\mathit{W}_2 \in \mathbb{R}^{4 \times d_{model}}$ for two linear transformations (there's a ReLU in the middle, which is small)
- Flop count: $2 \cdot 8 \cdot d_{model}^2$

**Some other things**
- There are typically layernorm that happen after each attention, where the weights there are a vector of length $d_{model}$
- There's another linear layer and then a softmax that sits on top, which is our output (token) embedding or unembedding or de-embedding or $\text{embedding}^{-1}$
- The original transformer has a cosine absolute positional encoding scheme, which is an addition operation on the token embedding.

Adding up all the flops:
$$F = n_{layers} \cdot (2 \cdot 3 \cdot d_{model}^2 + 2 \cdot d_{model}^2 + 16 \cdot d_{model}^2) = n_{layers} \cdot 24 \cdot d_{model}^2$$

Subbing in our 8192 model, we should get about 100B flops;
$$F = 64 \cdot 24 \cdot 8192^2 = 103079215104 \ \text{flops}$$

103079215104 over two is about 51.5B. We're a lil under (we get 51.5B instead of 52B) but that's because token (un)embeddings are nearly a billion parameters. It would be reasonable to do the latency calculations with $2 \cdot 12 \cdot n_{layers} \cdot  d_{model}^2$ instead $2 \cdot P$, but it's less than a 2% difference.

What about the the calculation of $z$ and all the other steps I didn't count? Those are all vector-vector (or even vector-scalar) operations, so they are built around a factor of $d_{model}$ rather than $d_{model}^2$. Even if we had 100 of these operations per layer, it would come out to a hundred million flops, which is 0.1% of the number of flops we counted.
